## Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## データセットの準備

In [ ]:
# kaggle ライブラリのインストール
!pip install kaggle

# 一時フォルダに .kaggleフォルダを作成
!mkdir ~/.kaggle

# MyDrive の kaggle.json を一時フォルダ内の .kaggleフォルダにコピー
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/

# アクセス権限の設定
!chmod 600 ~/.kaggle/kaggle.json

!mkdir ~/.kaggle

# zipファイルのダウンロード
!kaggle datasets download -d ashaheedq/video-games-sales-2019 -p /content/drive/MyDrive/kaggle

# 解凍
!unzip /content/drive/MyDrive/kaggle/video-games-sales-2019.zip -d /content/drive/MyDrive/kaggle

!rm /content/drive/MyDrive/kaggle/video-games-sales-2019.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/3.98M [00:00<?, ?B/s]
100% 3.98M/3.98M [00:00<00:00, 64.8MB/s]
Archive:  /content/drive/MyDrive/kaggle/video-games-sales-2019.zip
replace /content/drive/MyDrive/kaggle/vgsales-12-4-2019-short.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: None


## CUDA のバージョン変更

In [ ]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-10.1/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-10.1/lib64:{ld}"
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


## データの読込

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/kaggle/vgsales-12-4-2019.csv")
df.head(3)

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,NaN,40.24,NaN,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg


## データ拡張

In [ ]:
!pip install japanize-matplotlib
!pip install tgan

!pip uninstall tensorflow
!pip install "tensorflow-gpu>=1.13.0,<2.0"

None
     |████████████████████████████████| 4.1 MB 7.7 MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120274 sha256=1ffb28bbe5d6417630bbb8ad06bcf7ad063de122f6e844c96d3d36b20d73ce3e
  Stored in directory: /root/.cache/pip/wheels/83/97/6b/e9e0cde099cc40f972b8dd23367308f7705ae06cd6d4714658
Successfully built japanize-matplotlib
     |████████████████████████████████| 110.5 MB 1.2 MB/s 
     |████████████████████████████████| 273 kB 67.1 MB/s 
     |████████████████████████████████| 50 kB 9.1 MB/s 
     |████████████████████████████████| 503 kB 67.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 24.0 MB/s 
     |████████████████████████████████| 3.8 MB 58.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=4cfad03b473160e80c1213ec545c31d8e67b050d171504ce0e36d42034114d7e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a8

     |████████████████████████████████| 15.7 MB 12.4 MB/s 
     |████████████████████████████████| 9.5 MB 52.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 296 kB 56.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency 

Found existing installation: tensorflow 1.15.5
Uninstalling tensorflow-1.15.5:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-1.15.5.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
    /usr/local/lib/python3.7/dist-packages/tensorflow_core/*
Proceed (y/n)? Y
  Successfully uninstalled tensorflow-1.15.5
     |████████████████████████████████| 411.0 MB 25 kB/s 
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

## TGAN

In [ ]:
import os
import time
import random
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
from natsort import natsorted 

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import tensorflow
from tgan.data import load_demo_data
from tgan.model import TGANModel

import warnings
warnings.simplefilter('ignore')

In [ ]:
# シード値の固定
SEED = 42

random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
                                                    df.dropna(subset=["Global_Sales"]).drop(["Global_Sales",  "NA_Sales", "PAL_Sales", "JP_Sales", "Other_Sales"], axis=1), 
                                                    df.dropna(subset=["Global_Sales"])["Global_Sales"],  
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=SEED
                                                    ) 

In [ ]:
### パイプライン

# 数値データカラム名を取得
number_columns = X_train.select_dtypes(include="number").columns
number_columns = list(number_columns)

# カテゴリデータカラム名を取得
category_columns = X_train.select_dtypes(include="object").columns
category_columns = list(category_columns)

# カテゴリデータカラムの各カラムのカテゴリーの数を取得
category_unique_num = X_train.select_dtypes(include="object").nunique()

# 数値データ用の変換
numeric_transformer = Pipeline(steps=[
    ('num_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())]
)

# カテゴリーのエンコーディング法則を指定する
ordinal_all_cols_mapping = []

for column in df.select_dtypes(include="object"):
    ordinal_one_cols_mapping = []
    for category in natsorted(X_train[column].unique()):
        ordinal_one_cols_mapping.append(category)

    ordinal_all_cols_mapping.append(ordinal_one_cols_mapping)


# カテゴリー数が少ないカテゴリーデータ用の変換
categorical_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder(
                handle_unknown = 'use_encoded_value', # 未知数をunknown valueに置き換える設定
                unknown_value = -1,
                categories = ordinal_all_cols_mapping
            )
        )
    ]
)

# ColumnTransformerの作成
columns_transformers = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, number_columns),
        ('category', categorical_transformer, category_columns),
    ]
)

# カラム変換用のパイプラインの作成
transformer = Pipeline(
    [
        ("columns_transformers", columns_transformers),
    ]
)

# パイプラインの作成
pipe = Pipeline(
    [
        ("columns_transformers", columns_transformers),
        ('model', lgb.LGBMRegressor(random_state=42))
     ]
)

In [ ]:
# 学習・推論
gkf = GroupKFold(n_splits=5)

groups = X_train["Genre"]

best_params, history = {}, []

cv_result = []

pred_df = pd.DataFrame()

for i, (train_index, test_index) in enumerate(gkf.split(X_train, y_train, groups)):
    X_train_gkf, X_test_gkf = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_gkf, y_test_gkf = y_train.iloc[train_index], y_train.iloc[test_index]

    # 学習、推論
    pipe.fit(X_train_gkf, y_train_gkf)

    y_pred = pipe.predict(X_test_gkf)

    rmse = mean_squared_error(y_test_gkf, y_pred, squared=False)
    cv_result.append(rmse)

    pred = pipe.predict(X_test)

    pred_df[i] = pred

print("RMSE:", cv_result)
print("RMSE:", np.mean(cv_result))

RMSE: [0.3399196410524754, 0.11054671476889413, 0.1515846494616466, 0.028296266675878243, 0.33816232288908127]
RMSE: 0.19370191896959513


In [ ]:
pred_df.head()

,0,1,2,3,4
0,0.079978,0.080477,0.078200,0.087157,0.079929
1,0.020041,0.020541,0.020097,0.017319,0.020065
2,2.685723,2.649113,2.967197,2.667069,2.586828
3,0.516042,0.513283,0.514769,0.512154,0.511240
4,0.209935,0.210189,0.215592,0.210763,0.210023


In [ ]:
rmse_test = mean_squared_error(y_test, pred_df.mean(axis=1), squared=False)
print(rmse_test)

0.2390859888186454


## TGAN

In [ ]:
all_columns = X_test.columns.tolist()

### Nullのみで削除されるカラムを削除
print(len(all_columns))

for column in ["VGChartz_Score", "Total_Shipped"]:
    all_columns.remove(column)

print(len(all_columns))

18
16


In [ ]:
X_train_tf = transformer.fit_transform(X_train)
X_train_tf = pd.DataFrame(
                          X_train_tf,
                          columns = all_columns
                        )

X_test_tf = transformer.fit_transform(X_test)
X_test_tf = pd.DataFrame(
                          X_test_tf,
                          columns = all_columns
                        )

In [ ]:
train_df = pd.concat([X_train_tf, y_train], axis=1)

In [ ]:
continuous_value_columns = ["Rank", "Critic_Score", "User_Score", "Total_Shipped", "Global_Sales", "Year"]

tgan = TGANModel(
    continuous_value_columns,
    max_epoch=5,
    steps_per_epoch=5000,
    batch_size=100,
    )

tgan.fit(train_df)




[1221 12:02:05 @input_source.py:222] Setting up the queue 'QueueInput/input_queue' for CPU prefetching ...








Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[1221 12:02:05 @registry.py:126] gen/LSTM/00/FC input: [100, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[1221 12:02:05 @registry.py:134] gen/LSTM/00/FC output: [100, 100]
[1221 12:02:05 @registry.py:126] gen/LSTM/00/FC2 input: [100, 100]
[1221 12:02:05 @registry.py:134] gen/LSTM/00/FC2 output: [100, 13591]
[1221 12:02:05 @registry.py:126] gen/LSTM/00/FC3 input: [100, 13591]
[1221 12:02:05 @registry.py:134] gen/LSTM/00/FC3 output: [100, 100]
[1221 12:02:05 @registry.py:126] gen/LSTM/01/FC input: [100, 100]
[1221 12:02:05 @r

100%|#########9|4992/5000[02:54<00:00,30.99it/s]

100%|##########|5000/5000[02:57<00:00,28.21it/s]

[1221 12:05:20 @base.py:285] Epoch 1 (global_step 5000) finished, time:2 minutes 57 seconds.



[1221 12:05:21 @saver.py:79] Model saved to output/model/model-5000.
[1221 12:05:21 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.9
[1221 12:05:21 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.1
[1221 12:05:21 @monitor.py:467] GAN_loss/discrim/loss: 0.65898
[1221 12:05:21 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0278
[1221 12:05:21 @monitor.py:467] GAN_loss/gen/klloss: 0.088894
[1221 12:05:21 @monitor.py:467] GAN_loss/gen/loss: 0.9389
[1221 12:05:21 @monitor.py:467] QueueInput/queue_size: 50
[1221 12:05:21 @base.py:275] Start Epoch 2 ...


...

[1221 12:13:27 @base.py:285] Epoch 4 (global_step 20000) finished, time:2 minutes 41 seconds.


[1221 12:13:27 @saver.py:79] Model saved to output/model/model-20000.
[1221 12:13:27 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.98
[1221 12:13:27 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.09
[1221 12:13:27 @monitor.py:467] GAN_loss/discrim/loss: 0.62532
[1221 12:13:27 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.0665
[1221 12:13:27 @monitor.py:467] GAN_loss/gen/klloss: 0.059515
[1221 12:13:27 @monitor.py:467] GAN_loss/gen/loss: 1.0069
[1221 12:13:27 @monitor.py:467] QueueInput/queue_size: 50
[1221 12:13:27 @base.py:275] Start Epoch 5 ...


100%|##########|5000/5000[02:41<00:00,31.04it/s]

[1221 12:16:08 @base.py:285] Epoch 5 (global_step 25000) finished, time:2 minutes 41 seconds.


[1221 12:16:09 @saver.py:79] Model saved to output/model/model-25000.
[1221 12:16:09 @monitor.py:467] GAN_loss/discrim/accuracy_fake: 0.95
[1221 12:16:09 @monitor.py:467] GAN_loss/discrim/accuracy_real: 0.16
[1221 12:16:09 @monitor.py:467] GAN_loss/discrim/loss: 0.6384
[1221 12:16:09 @monitor.py:467] GAN_loss/gen/final-g-loss: 1.034
[1221 12:16:09 @monitor.py:467] GAN_loss/gen/klloss: 0.07838
[1221 12:16:09 @monitor.py:467] GAN_loss/gen/loss: 0.9556
[1221 12:16:09 @monitor.py:467] QueueInput/queue_size: 50
[1221 12:16:09 @base.py:289] Training has finished!

[1221 12:16:09 @input_source.py:178] EnqueueThread QueueInput/input_queue Exited.

[1221 12:16:11 @collection.py:146] New collections created in tower : tf.GraphKeys.REGULARIZATION_LOSSES
[1221 12:16:11 @collection.py:165] These collections were modified but restored in : (tf.GraphKeys.SUMMARIES: 0->2)


[1221 12:16:11 @sessinit.py:87] WRN The following variables are in the checkpoint, but not found in the graph: global_step, optim

In [ ]:
num_samples = 50000

train_sample = tgan.sample(num_samples)

print(train_sample.shape)

train_sample.head(3)

|          |499/?[00:07<00:00,71.21it/s]

(50000, 17)


,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,Critic_Score,User_Score,Year,Last_Update,url,status,Vgchartzscore,img_url,Global_Sales
0,-0.8302233283072288,0.07904834342215986,0.022465580452656453,-1.912342755368196,0.0,0.03376941899612632,5339.0,5636.0,18.0,1.0,20.0,nan,1850.0,233.0,12459.0,6249.0,nan
1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.02
2,nan,0.07904834342215986,0.022465580452656453,-1.912342755368196,0.0,0.03376941899612632,5339.0,5636.0,5.0,1.0,20.0,77.0,1850.0,233.0,12459.0,6249.0,nan


In [ ]:
train_sample_fillna = train_sample.astype(float)

train_sample_fillna = train_sample_fillna.fillna(method='ffill')

train_sample_fillna = train_sample_fillna.dropna(axis=0)

train_sample_fillna = train_sample_fillna.reset_index(drop=True)

X_train_sample_fillna,  y_train_sample_fillna = train_sample_fillna.drop(["Global_Sales"], axis=1), train_sample_fillna["Global_Sales"]

In [ ]:
print(X_train[all_columns].shape, y_train.shape)

print(X_train_sample_fillna.shape, y_train_sample_fillna.shape)

(13590, 16) (13590,)
(49998, 16) (49998,)


In [ ]:
X_train_comb = pd.concat([
           X_train_tf[all_columns],
           X_train_sample_fillna
]).reset_index(drop=True)

y_train_comb = pd.concat([
           y_train,
           y_train_sample_fillna
]).reset_index(drop=True)

In [ ]:
# 学習・推論
gkf = GroupKFold(n_splits=5)

groups = X_train_comb["Genre"]

cv_result_tgan = []

pred_df = pd.DataFrame()

model = lgb.LGBMRegressor(random_state=42)

for i, (train_index, test_index) in enumerate(gkf.split(X_train_comb, y_train_comb, groups)):
    X_train_gkf, X_test_gkf = X_train_comb.iloc[train_index], X_train_comb.iloc[test_index]
    y_train_gkf, y_test_gkf = y_train_comb.iloc[train_index], y_train_comb.iloc[test_index]

    # 学習、推論
    model.fit(X_train_gkf, y_train_gkf)

    y_pred = model.predict(X_test_gkf)

    rmse = mean_squared_error(y_test_gkf, y_pred, squared=False)
    cv_result_tgan.append(rmse)

    pred = pipe.predict(X_test)

    pred_df[i] = pred

print("RMSE:", cv_result_tgan)
print("RMSE:", np.mean(cv_result_tgan))

RMSE: [0.08032248007324973, 0.03429247024718503, 0.08799995748240287, 0.04285124463257716, 0.07190888337249346]
RMSE: 0.06347500716158165


In [ ]:
rmse_test_tgan = mean_squared_error(y_test, pred_df.mean(axis=1), squared=False)
print(rmse_test_tgan)

0.28956467284291515


In [ ]:
print("RMSE:", round(rmse_test, 3))
print("T-GAN によるデータ拡張 RMSE:", round(rmse_test_tgan, 3))

RMSE: 0.239
T-GAN によるデータ拡張 RMSE: 0.29
